# Regressão Linear Múltipla

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
%matplotlib inline 

In [2]:
!ls

README.md
RLM-01.png
RLM-02.png
RegressaoLinearMultipla-03-Exercicio.ipynb
RegressaoLinearMultipla-03.ipynb
RegressaoLinearMultipla-04-ComEscala-EXERCICIO-01-Copy1.ipynb
RegressaoLinearMultipla-04-ComEscala-EXERCICIO-01.ipynb
RegressaoLinearMultipla-04-ComEscala.ipynb
RegressaoLinearMultipla-04.ipynb
SelecaoDeAtributos-v03.ipynb
SelecaoDeAtributos.ipynb
Untitled.ipynb
auto-mpg.data
auto-mpg.data-original
auto-mpg.names
breast-cancer-wisconsin.data
breast-cancer-wisconsin.names


OBS: Escolher um dataset (arquivo) no Site do UCI: <br>
https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table

In [3]:
df = pd.read_csv('breast-cancer-wisconsin.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


 #  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)

8. Missing attribute values: 16

   There are 16 instances in Groups 1 to 6 that contain a single missing 
   (i.e., unavailable) attribute value, now denoted by "?".  


In [4]:
atributos = [ 'Sample code number', 'Clump Thickness','Uniformity of Cell Size'
             ,'Uniformity of Cell Shape', 'Marginal Adhesion','Single Epithelial Cell Size',
             'Bare Nuclei', 'Bland Chromatin','Normal Nucleoli','Mitoses', 'Class']


In [5]:
df.columns = atributos
df.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [11]:
atributos[1:]

['Clump Thickness',
 'Uniformity of Cell Size',
 'Uniformity of Cell Shape',
 'Marginal Adhesion',
 'Single Epithelial Cell Size',
 'Bare Nuclei',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses',
 'Class']

In [12]:
df = df.loc[:, atributos[1:]]
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
Clump Thickness                699 non-null int64
Uniformity of Cell Size        699 non-null int64
Uniformity of Cell Shape       699 non-null int64
Marginal Adhesion              699 non-null int64
Single Epithelial Cell Size    699 non-null int64
Bare Nuclei                    699 non-null object
Bland Chromatin                699 non-null int64
Normal Nucleoli                699 non-null int64
Mitoses                        699 non-null int64
Class                          699 non-null int64
dtypes: int64(9), object(1)
memory usage: 54.7+ KB


In [16]:
#df['Bare Nuclei'] = df['Bare Nuclei'].astype(int)
df['Bare Nuclei'] = df['Bare Nuclei'].apply(lambda x: -1 if x == '?' else x)
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [17]:
df['Bare Nuclei'].value_counts()

1     402
10    132
5      30
2      30
3      28
8      21
4      19
-1     16
9       9
7       8
6       4
Name: Bare Nuclei, dtype: int64

In [18]:
dataset = df.copy()

In [ ]:
from sklearn import preprocessing
def converte_categoria_para_inteiro(valores):
    le = preprocessing.LabelEncoder()
    inteiros = le.fit_transform(valores)
    print (set(le.inverse_transform(inteiros)))
    return le, inteiros

#obj, df['buying'] = converte_categoria_para_inteiro(df['buying'])
#df.head()

# converter todos os atributos categóricos
objetos = []
atributos = ['atributo1', 'atributo2', 'atributo3']
for atributo in atributos:
    obj, df[atributo] = converte_categoria_para_inteiro(df[atributo])
    objetos.append(obj)
df.head()
dataset = df.copy()

In [19]:
dataset = df.copy()
len(dataset)

699

## Análise Descritiva

In [20]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Clump Thickness,699.0,4.417740,2.815741,1.0,2.0,4.0,6.0,10.0
Uniformity of Cell Size,699.0,3.134478,3.051459,1.0,1.0,1.0,5.0,10.0
Uniformity of Cell Shape,699.0,3.207439,2.971913,1.0,1.0,1.0,5.0,10.0
Marginal Adhesion,699.0,2.806867,2.855379,1.0,1.0,1.0,4.0,10.0
Single Epithelial Cell Size,699.0,3.216023,2.214300,1.0,2.0,2.0,4.0,10.0
Bland Chromatin,699.0,3.437768,2.438364,1.0,2.0,3.0,5.0,10.0
Normal Nucleoli,699.0,2.866953,3.053634,1.0,1.0,1.0,4.0,10.0
Mitoses,699.0,1.589413,1.715078,1.0,1.0,1.0,1.0,10.0
Class,699.0,2.689557,0.951273,2.0,2.0,2.0,4.0,4.0


In [21]:
dataset.describe()['Class'] # variável preditora ou Classe ou Label

count    699.000000
mean       2.689557
std        0.951273
min        2.000000
25%        2.000000
50%        2.000000
75%        4.000000
max        4.000000
Name: Class, dtype: float64

In [22]:
dataset['Class'].value_counts()

2    458
4    241
Name: Class, dtype: int64

In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
Clump Thickness                699 non-null int64
Uniformity of Cell Size        699 non-null int64
Uniformity of Cell Shape       699 non-null int64
Marginal Adhesion              699 non-null int64
Single Epithelial Cell Size    699 non-null int64
Bare Nuclei                    699 non-null object
Bland Chromatin                699 non-null int64
Normal Nucleoli                699 non-null int64
Mitoses                        699 non-null int64
Class                          699 non-null int64
dtypes: int64(9), object(1)
memory usage: 54.7+ KB


## Gerando número de observações 

In [ ]:
observations = len(dataset)
observations

In [24]:
dataset.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [25]:
atributosX = ['Clump Thickness','Uniformity of Cell Size'
             ,'Uniformity of Cell Shape', 'Marginal Adhesion','Single Epithelial Cell Size',
             'Bare Nuclei', 'Bland Chromatin','Normal Nucleoli','Mitoses']
atributosX

['Clump Thickness',
 'Uniformity of Cell Size',
 'Uniformity of Cell Shape',
 'Marginal Adhesion',
 'Single Epithelial Cell Size',
 'Bare Nuclei',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses']

## Coletando x e y

In [26]:
X = dataset.loc[:, atributosX  ]
y = dataset['Class'].values

In [27]:
X.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1


In [28]:
y[:5]

array([2, 2, 2, 2, 2])

### Matriz de Correlação

In [ ]:
# Gerando a matriz
X = dataset.iloc[:,:-1]
matriz_corr = X.corr()
print (matriz_corr)

In [ ]:
# matriz de Correlação com a variável preditora
mt = pd.DataFrame(dataset.values, columns=dataset.columns)
mt_corr = mt.corr()
print (abs(mt_corr['mpg']).sort_values(ascending=False))

## Visualizando a matriz de correlação (somente os atributos)

In [ ]:
# Criando um Correlation Plot
def visualize_correlation_matrix(data, hurdle = 0.0):
    fig = plt.figure(figsize=(12,9))
    ax = fig.add_subplot(111)
    R = np.corrcoef(data, rowvar = 0)
    R[np.where(np.abs(R) < hurdle)] = 0.0
    heatmap = plt.pcolor(R, cmap = mpl.cm.coolwarm, alpha = 0.8)
    heatmap.axes.set_frame_on(False)
    heatmap.axes.set_yticks(np.arange(R.shape[0]) + 0.5, minor = False)
    heatmap.axes.set_xticks(np.arange(R.shape[1]) + 0.5, minor = False)
    heatmap.axes.set_xticklabels(dataset.columns[:-1], minor = False)
    plt.xticks(rotation=90)
    heatmap.axes.set_yticklabels(dataset.columns[:-1], minor = False)
    plt.tick_params(axis = 'both', which = 'both', bottom = 'off', top = 'off', left = 'off', right = 'off')   
    plt.colorbar()
    plt.show()

In [ ]:
# Visualizando o Plot
visualize_correlation_matrix(X, hurdle = 0.5)

## Feature Scaling

In [ ]:
X.head()

Podemos aplicar Feature Scaling através de Padronização ou Normalização. <br />Normalização aplica escala aos dados com intervalos entre 0 e 1. <br />A Padronização divide a média pelo desvio padrão para obter uma unidade de variância. <br />Vamos usar a Padronização (StandardScaler) pois nesse caso esta técnica ajusta os coeficientes e torna a superfície de erros mais "tratável".

### Aplicando Padronização

In [ ]:
from sklearn.preprocessing import StandardScaler
standardization = StandardScaler()
Xst = standardization.fit_transform(X)

original_means = standardization.mean_
originanal_stds = standardization.scale_
print('Dataset Original')
X.head()

In [ ]:
print('Dataset Padronizado')
dstd = pd.DataFrame(Xst,columns=['cylinders', 'displacement', 'horsepower', 'weight',
                     'acceleration', 'model year', 'origin'])
dstd.head()

In [ ]:
X.max()

In [ ]:
dstd.max()

In [ ]:
X.min()

In [ ]:
dstd.min()

In [ ]:
y[:5]

### Desfazendo a Padronização

In [ ]:
dfXst = pd.DataFrame(Xst, columns=['cylinders', 'displacement', 'horsepower', 'weight',
                     'acceleration', 'model year', 'origin'])
dfXst.head()

In [ ]:
Xinv = standardization.inverse_transform(Xst)
dfinverser = pd.DataFrame(Xinv, columns=['cylinders', 'displacement', 'horsepower', 'weight',
                     'acceleration', 'model year', 'origin'])
dfinverser.head()

In [ ]:
dataset.head()

# Criar o modelo

In [29]:
# Criando um modelo
modelo = linear_model.LinearRegression() #normalize = False, fit_intercept = True)
modelo.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [36]:
# Criando um modelo
#from sklearn.svm import SVR
#modelo = SVR() #normalize = False, fit_intercept = True)
#modelo.fit(X,y)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

## Mostrar os pesos dos Atributos ( yprev = w1X1 + w2X2 + ... + wnXn )

In [30]:
#atributosX 
for coef, var in sorted(zip(modelo.coef_, atributosX), reverse = True):
    print ("%6.3f %s" % (coef,var))

 0.090 Bare Nuclei
 0.064 Clump Thickness
 0.044 Uniformity of Cell Size
 0.042 Bland Chromatin
 0.036 Normal Nucleoli
 0.035 Uniformity of Cell Shape
 0.014 Single Epithelial Cell Size
 0.011 Marginal Adhesion
 0.004 Mitoses


### Avaliando o modelo com o R Squared (R²) 

In [ ]:
def r2_est(X,y):
    modelo = linear_model.LinearRegression(normalize = False, fit_intercept = True)
    return r2_score(y, modelo.fit(X,y).predict(X))

In [ ]:
print ('R2: %0.3f' %  r2_est(Xst,y))

### Gera o impacto de cada atributo no R² 

In [ ]:
r2_impact = list()
for j in range(X.shape[1]):
    selection = [i for i in range(X.shape[1]) if i!=j]
    r2_impact.append(((r2_est(X,y) - r2_est(X.values[:,selection],y)), dataset.columns[j]))
    
for imp, varname in sorted(r2_impact, reverse = True):
    print ('%6.3f %s' %  (imp, varname))

# Fazer Previsões

In [ ]:
dataset.tail()

In [ ]:
#             CRIM	  ZN  INDUS	  CHAS	     NOX      RM	 AGE	DIS	  RAD	  TAX	 PTRATIO	B	   LSTAT
Xteste = [   30.,   0.0, 11,      0.0,      0.5,     6. ,   80.2,  2.3,  1.0,    273.0,   21.2,      396.1,  7.4
    ]

standardization = StandardScaler()
Xteste_escala = standardization.fit_transform(np.array(Xteste).reshape(1, -1)) 
#m = modelo.fit(X,y)
#Xteste_norm = standardization.fit_transform(X)
#modelo.predict(np.array(Xteste).reshape(1, -1))[0]

modelo.predict(Xteste_escala)[0]

In [ ]:
Xteste = [
    [   0.02,   0.2, 7.01,   0.0,   0.5,  7.1,  45.2,  6.1,  3.0,    222,   15.2,      396.1,   5.4],
    [   0.01,   0.1,11.01,   0.0,   0.6,  6.1,  80.2,  2.5,  1.0,    273,   21.2,      396.9,   12.6],
]


Xteste_escala = standardization.fit_transform(np.array(Xteste)) 

modelo.predict(np.array(Xteste_escala))

# Métricas para Algoritmos de Regressão

### Gerando o dataset

In [ ]:
dataset['y_prev'] = modelo.predict(Xst)
#dataset['Erro'] = dataset['y_prev'] - dataset['target'] 
dataset.head()

In [ ]:
dataset['Erro'] = abs (dataset['y_prev'] - dataset['target'] )
dataset.head()

In [ ]:
dataset.Erro.sum()

## MAE - Mean Absolute Error
É a soma da diferença absoluta entre previsões e valores reais.<br />
Fornece uma ideia de quão erradas estão nossas previsões.<br />
Valor igual a 0 indica que não há erro, sendo a previsão perfeita <br />
(a exemplo do Logloss, a função cross_val_score inverte o valor) 

In [ ]:
MAE: -0.263 (0.075)

In [37]:
from sklearn import model_selection
kfold = 10
resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, scoring = 'neg_mean_absolute_error')

# Print do resultado
print("MAE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

MAE: -0.219 (0.071)


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

## MSE - Mean Squared Error
Similar ao MAE, fornece a magnitude do erro do modelo.<br />
Ao extrairmos a raiz quadrada do MSE convertemos as unidades de volta ao original, 
o que pode ser útil para descrição e apresentação.<br />
Isso é chamado RMSE (Root Mean Squared Error)

In [ ]:
MSE: -0.155 (0.049)

In [38]:
# Definindo os valores para o número de folds
num_folds = 10
num_instances = len(X)
seed = 7

# Separando os dados em folds
kfold = model_selection.KFold(num_folds, True, random_state = seed)

resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, scoring = 'neg_mean_squared_error')

# Print do resultado
print("MSE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

MSE: -0.139 (0.055)


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

# RMSE (Root Mean Squared Error)

Similar ao MAE, fornece a magnitude do erro do modelo.<br>
Ao extrairmos a raiz quadrada do MSE convertemos as unidades de volta ao original, o que pode ser útil para descrição e apresentação.

In [ ]:
RMSE: 0.393

In [39]:
from math import sqrt
print("RMSE: %.3f " % (sqrt(abs(resultado.mean()))))

RMSE: 0.373 


## R2  
Essa métrica fornece uma indicação do nível de precisão das previsões em relação aos valores observados.<br />
Também chamado de coeficiente de determinação.<br />
Valores entre 0 e 1, sendo 1 o valor ideal.

In [ ]:
R^2: 0.828 (0.055)

In [40]:
resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, scoring = 'r2')

# Print do resultado
print("R^2: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

R^2: 0.845 (0.063)


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 